In [ ]:
pip install pandas requests


In [ ]:
import pandas as pd
import requests

In [ ]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
df = pd.read_json(url)


In [ ]:
print(df. head())

print(df.info())

print(df.describe())

   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '...   
2  {'InternetService': 'Fiber optic', 'OnlineSecu...   
3  {'I

In [ ]:
# Paso 1: Detectar las columnas que contienen diccionarios
cols_to_flatten = [col for col in df.columns if isinstance(df[col].iloc[0], dict)]

print("Columnas a desanidar:", cols_to_flatten)


Columnas a desanidar: ['customer', 'phone', 'internet', 'account']


In [ ]:
# Paso 2: Crear un DataFrame base con las columnas no anidadas
df_base = df.drop(columns=cols_to_flatten).copy()

# Paso 3: Desanidar cada columna detectada automáticamente
flattened_parts = [pd.json_normalize(df[col]) for col in cols_to_flatten]

# Paso 4: Unir todo en un solo DataFrame
df_final = pd.concat([df_base] + flattened_parts, axis=1)

# Paso 5: Limpiar nombres de columnas
df_final.columns = df_final.columns.str.strip().str.lower().str.replace(' ', '_')

# Vista previa
df_final.head()


,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges.monthly,charges.total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [15]:
# 1. Valores ausentes por columna
print("Valores ausentes por columna:")
print(df_final.isnull().sum())
print("\n" + "-"*50 + "\n")

# 2. Filas duplicadas
num_duplicados = df_final.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}\n")

if num_duplicados > 0:
    df_final = df_final.drop_duplicates()
    print("Duplicados eliminados.\n")
print("-"*50 + "\n")

# 3. Errores de formato en 'charges.total'
df_final['charges.total'] = pd.to_numeric(df_final['charges.total'], errors='coerce')
num_null_totalcharges = df_final['charges.total'].isnull().sum()
print(f"Valores NaN en 'charges.total' tras conversión a numérico: {num_null_totalcharges}\n")
print("-"*50 + "\n")

# 4. Valores únicos en columnas categóricas para detectar inconsistencias
categorical_cols = [
    'gender', 'partner', 'dependents', 'phoneservice', 'multiplelines',
    'internetservice', 'onlinesecurity', 'techsupport', 'contract',
    'paymentmethod', 'paperlessbilling', 'churn'
]

for col in categorical_cols:
    unique_vals = df_final[col].unique()
    print(f"Valores únicos en '{col}': {unique_vals}\n")

print("-"*50 + "\n")

# Opcional: resumen general del DataFrame
print("Resumen general del DataFrame:")
print(df_final.info())



Valores ausentes por columna:
customerid           0
churn                0
gender               0
seniorcitizen        0
partner              0
dependents           0
tenure               0
phoneservice         0
multiplelines        0
internetservice      0
onlinesecurity       0
onlinebackup         0
deviceprotection     0
techsupport          0
streamingtv          0
streamingmovies      0
contract             0
paperlessbilling     0
paymentmethod        0
charges.monthly      0
charges.total       11
dtype: int64

--------------------------------------------------

Número de filas duplicadas: 0

--------------------------------------------------

Valores NaN en 'charges.total' tras conversión a numérico: 11

--------------------------------------------------

Valores únicos en 'gender': ['Female' 'Male']

Valores únicos en 'partner': ['Yes' 'No']

Valores únicos en 'dependents': ['Yes' 'No']

Valores únicos en 'phoneservice': ['Yes' 'No']

Valores únicos en 'multiplelines': ['No

In [16]:
# Ver cuántas filas tienen churn vacío
print("Filas con churn vacío:", df_final[df_final['churn'].str.strip() == ''].shape[0])

# Eliminar esas filas
df_final = df_final[df_final['churn'].str.strip() != '']


Filas con churn vacío: 224


In [18]:
#eliminar valores vacios en charges total
df_final = df_final[df_final['charges.total'].notna()]


In [19]:
# Columnas relacionadas con teléfono e internet
columns_to_clean = [
    'multiplelines', 'onlinesecurity', 'onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies'
]

# Reemplazar 'No phone service' y 'No internet service' por 'No'
for col in columns_to_clean:
    df_final[col] = df_final[col].replace({
        'No phone service': 'No',
        'No internet service': 'No'
    })


In [20]:
print(df_final.isnull().sum())
print(df_final['churn'].unique())


customerid          0
churn               0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
charges.monthly     0
charges.total       0
dtype: int64
['No' 'Yes']


In [21]:
# Revisión general del DataFrame en una sola celda
print("✅ Valores nulos por columna:\n", df_final.isnull().sum(), "\n" + "-"*50)

print("✅ Tipos de datos por columna:\n", df_final.dtypes, "\n" + "-"*50)

print("✅ Resumen estadístico de columnas numéricas:\n", df_final.describe(), "\n" + "-"*50)

print("✅ Distribución de la variable 'churn':\n", df_final['churn'].value_counts(), "\n" + "-"*50)

print("✅ Valores únicos por columna categórica:")
for col in df_final.select_dtypes(include='object').columns:
    print(f"- {col}: {df_final[col].unique()}")



✅ Valores nulos por columna:
 customerid          0
churn               0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
charges.monthly     0
charges.total       0
dtype: int64 
--------------------------------------------------
✅ Tipos de datos por columna:
 customerid           object
churn                object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv         

In [23]:
df_final = df_final.convert_dtypes()

print(df_final.dtypes)


customerid          string[python]
churn               string[python]
gender              string[python]
seniorcitizen                Int64
partner             string[python]
dependents          string[python]
tenure                       Int64
phoneservice        string[python]
multiplelines       string[python]
internetservice     string[python]
onlinesecurity      string[python]
onlinebackup        string[python]
deviceprotection    string[python]
techsupport         string[python]
streamingtv         string[python]
streamingmovies     string[python]
contract            string[python]
paperlessbilling    string[python]
paymentmethod       string[python]
charges.monthly            Float64
charges.total              Float64
dtype: object
